# Grouping for Aggregation, Filtration, and Transformation

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 15, 'display.max_rows', 10, 'display.max_colwidth', 12)

## Introduction

* 데이터분석의 기본 : Split - Apply - Combine (분할 - 적용 - 병합)
* Pandas에서 제공하는 .groupby 메서드를 활용하면 데이터를 다양한 방식으로 그룹화하고 각 그룹에 독립적인 함수를 적용한 다음 단일 데이터셋을 반환할 수 있음
---
df.groupby(['list', 'of', 'grouping', 'columns'])  
df.groupby('single_column')   

In [2]:
# .groupby 메서드 호출 결과는 groupby 객체, 다른 메서드 처럼 직관적이지는 않으며
# 메서드를 체인시켜야 그룹화 결과를 확인할 수 있다.
flights = pd.read_csv('data/flights.csv')
flights.groupby('MONTH')

### Defining an Aggregation

* .groupby 메서드의 가장 일반적인 용도는 집계(aggregation)을 수행하는 것
* 집계 : 많은 입력 시퀀스를 요약하거나 결합해 단일값으로 출력하는것 
* ex) 열 전체 합산, 최대값 찾기, 분산 및 평균 구하기 등등

In [3]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [4]:
# .agg 메서드 : groupby 객체로 집계를 수행하기 위한 메서드 
# 집계 열과 집계 함수 쌍의 딕셔너리 형태 사용 예시
(flights
     .groupby('AIRLINE')
     .agg({'ARR_DELAY':'mean'})
)

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580
...,...
OO,7.593463
UA,7.765755
US,1.681105


In [5]:
# 집계 열을 인덱스 연산자에, 집계 함수를 .agg내에 str 형태로 전달하는 형태 예시 
(flights
     .groupby('AIRLINE')
     ['ARR_DELAY']
     .agg('mean')
)

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
        ...   
OO    7.593463
UA    7.765755
US    1.681105
VX    5.348884
WN    6.397353
Name: ARR_DELAY, Length: 14, dtype: float64

In [6]:
# 집계 열을 인덱스 연산자에, 집계 함수를 .agg에 집계함수를 직접 전달하는 형태 예시 
(flights
    .groupby('AIRLINE')
    ['ARR_DELAY']
    .agg(np.mean)
)

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
        ...   
OO    7.593463
UA    7.765755
US    1.681105
VX    5.348884
WN    6.397353
Name: ARR_DELAY, Length: 14, dtype: float64

In [7]:
# 집계 열을 인덱스 연산자에 넣고, .agg를 생략하고 집계함수를 직접 사용 가능
# 대부분의 집계함수를 이렇게 사용 가능하다
(flights
    .groupby('AIRLINE')
    ['ARR_DELAY']
    .mean()
)

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
        ...   
OO    7.593463
UA    7.765755
US    1.681105
VX    5.348884
WN    6.397353
Name: ARR_DELAY, Length: 14, dtype: float64

### There's more...

In [8]:
# .agg에 집계함수가 아닌 함수는 사용이 불가능한 예시
(flights
   .groupby('AIRLINE')
   ['ARR_DELAY']
   .agg(np.sqrt)
)

C:\Users\ZeusRyu\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


ValueError: Must produce aggregated value

## Grouping and aggregating with multiple columns and functions

#1 요일별 모든 항공사의 취소된 항공편 수   

In [9]:
flights['CANCELLED'].unique() # 취소된거는 1인가 보다

array([0, 1], dtype=int64)

In [10]:
(flights
    .groupby(['AIRLINE', 'WEEKDAY']) # '요일별' '항공사' 그룹화 
    ['CANCELLED']  # 취소 항공편을 집계할 열로 지정
    .agg('sum') # 취소 항공편이 1 의 값을 가지니 그 값들의 합을 모으면 원하는 결과 도출
)

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
                    ..
WN       3          18
         4          10
         5           7
         6          10
         7           7
Name: CANCELLED, Length: 98, dtype: int64

#2 요일별 모든 항공사의 취소 또는 우회한 항공편의 수와 비율   

In [11]:
(flights
    .groupby(['AIRLINE', 'WEEKDAY']) # 요일별 모든 항공사 그룹화
    ['CANCELLED', 'DIVERTED'] # 취소와 우회여부를 대해 집계할 열로 지정 
    .agg(['sum', 'mean']) # sum으로 취소, 우회 횟수를 구하고, mean으로 비율 구하여 결과 도출 
)

<ipython-input-11-6c86afd2dd2f>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  (flights


CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786
...                   ...       ...      ...       ...
WN      3              18  0.014118        2  0.001569
        4              10  0.007911        4  0.003165
        5               7  0.005828        0  0.000000
        6              10  0.010132        3  0.003040
        7               7  0.006066        3  0.002600

[98 rows x 4 columns]

#3 각 출발지와 도착지에 대해 총 항공편 수, 취소된 항공편의 수와 비율, 비행시간의 평균과 분산

In [12]:
(flights
    .groupby(['ORG_AIR', 'DEST_AIR']) # 출발지, 도착지 기준 그룹화
    .agg({'CANCELLED':['sum', 'mean', 'size'], # 그룹의 취소 열에 대한 합계, 평균, 크기(몇개의 행이 있는지)
          'AIR_TIME':['mean', 'var']}) # 비행시간 열에 대해 평균과 분산 계산
)

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
ATL     ABE              0  0.000000   31   96.387097   45.778495
        ABQ              0  0.000000   16  170.500000   87.866667
        ABY              0  0.000000   19   28.578947    6.590643
        ACY              0  0.000000    6   91.333333   11.466667
        AEX              0  0.000000   40   78.725000   47.332692
...                    ...       ...  ...         ...         ...
SFO     SNA              4  0.032787  122   64.059322   11.338331
        STL              0  0.000000   20  198.900000  101.042105
        SUN              0  0.000000   10   78.000000   25.777778
        TUS              0  0.000000   20  100.200000   35.221053
        XNA              0  0.000000    2  173.500000    0.500000

[1130 rows x 5 columns]

In [13]:
# 앞선 방법으로 다중 집계를 시도하면 결과 컬럼이 계층 구조로 만들어짐
# 이를 피하려면 다음과 같이 NamedAgg를 사용하는 방법도 가능
# '새로운 집계 열 정의' = pd.NamedAgg(column='집계 열', aggfunc='집계 함수')  
(flights
    .groupby(['ORG_AIR', 'DEST_AIR'])
    .agg(sum_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='sum'),
         mean_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='mean'),
         size_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='size'),
         mean_air_time=pd.NamedAgg(column='AIR_TIME', aggfunc='mean'),
         var_air_time=pd.NamedAgg(column='AIR_TIME', aggfunc='var'))
)

sum_cancelled  mean_cancelled  size_cancelled  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                 0       0.000000              31      
        ABQ                 0       0.000000              16      
        ABY                 0       0.000000              19      
        ACY                 0       0.000000               6      
        AEX                 0       0.000000              40      
...                       ...            ...             ...      
SFO     SNA                 4       0.032787             122      
        STL                 0       0.000000              20      
        SUN                 0       0.000000              10      
        TUS                 0       0.000000              20      
        XNA                 0       0.000000               2      

                  mean_air_time  var_air_time  
ORG_AIR DEST_AIR                               
ATL     ABE         96.387097      45.778495   
        ABQ        170.500000      87.866667   
        ABY         28.578947       6.590643   
        ACY         91.333333      11.466667   
        AEX         78.725000      47.332692   
...                       ...            ...   
SFO     SNA         64.059322      11.338331   
        STL        198.900000     101.042105   
        SUN         78.000000      25.777778   
        TUS        100.200000      35.221053   
        XNA        173.500000       0.500000   

[1130 rows x 5 columns]

In [14]:
# 혹은 계층 구조로 만들어진 컬럼을 .to_flat_index 메서드를 사용해 펼칠수 도 있음
res = (flights
    .groupby(['ORG_AIR', 'DEST_AIR'])
    .agg({'CANCELLED':['sum', 'mean', 'size'],
          'AIR_TIME':['mean', 'var']})
)
print(res.columns.to_flat_index()) 
res.columns = ['_'.join(x) for x in
    res.columns.to_flat_index()]
res

Index([ ('CANCELLED', 'sum'), ('CANCELLED', 'mean'), ('CANCELLED', 'size'),
        ('AIR_TIME', 'mean'),   ('AIR_TIME', 'var')],
      dtype='object')


CANCELLED_sum  CANCELLED_mean  CANCELLED_size  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                 0       0.000000              31      
        ABQ                 0       0.000000              16      
        ABY                 0       0.000000              19      
        ACY                 0       0.000000               6      
        AEX                 0       0.000000              40      
...                       ...            ...             ...      
SFO     SNA                 4       0.032787             122      
        STL                 0       0.000000              20      
        SUN                 0       0.000000              10      
        TUS                 0       0.000000              20      
        XNA                 0       0.000000               2      

                  AIR_TIME_mean  AIR_TIME_var  
ORG_AIR DEST_AIR                               
ATL     ABE         96.387097      45.778495   
        ABQ        170.500000      87.866667   
        ABY         28.578947       6.590643   
        ACY         91.333333      11.466667   
        AEX         78.725000      47.332692   
...                       ...            ...   
SFO     SNA         64.059322      11.338331   
        STL        198.900000     101.042105   
        SUN         78.000000      25.777778   
        TUS        100.200000      35.221053   
        XNA        173.500000       0.500000   

[1130 rows x 5 columns]

In [15]:
# 이것도 따로 처리하지 않고 한번에 체인 메서드로 묶어주고 싶다면
# pipe 메서드 사용가능
# pipe 메서드 -> DataFrame이나 Series를 넣으면 DataFrame이나 Series를 리턴하는 function을 적용시키는 메서드
def flatten_cols(df):
    df.columns = ['_'.join(x) for x in
        df.columns.to_flat_index()]
    return df

In [16]:
(flights
    .groupby(['ORG_AIR', 'DEST_AIR'])
    .agg({'CANCELLED':['sum', 'mean', 'size'],
          'AIR_TIME':['mean', 'var']})
    .pipe(flatten_cols)
)

CANCELLED_sum  CANCELLED_mean  CANCELLED_size  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                 0       0.000000              31      
        ABQ                 0       0.000000              16      
        ABY                 0       0.000000              19      
        ACY                 0       0.000000               6      
        AEX                 0       0.000000              40      
...                       ...            ...             ...      
SFO     SNA                 4       0.032787             122      
        STL                 0       0.000000              20      
        SUN                 0       0.000000              10      
        TUS                 0       0.000000              20      
        XNA                 0       0.000000               2      

                  AIR_TIME_mean  AIR_TIME_var  
ORG_AIR DEST_AIR                               
ATL     ABE         96.387097      45.778495   
        ABQ        170.500000      87.866667   
        ABY         28.578947       6.590643   
        ACY         91.333333      11.466667   
        AEX         78.725000      47.332692   
...                       ...            ...   
SFO     SNA         64.059322      11.338331   
        STL        198.900000     101.042105   
        SUN         78.000000      25.777778   
        TUS        100.200000      35.221053   
        XNA        173.500000       0.500000   

[1130 rows x 5 columns]

In [17]:
# 그룹화 하는 컬럼이 category 타입이라면 그룹화 하는 과정에서 모든 조합을 다 출력함
# ex) 아래 예시에선 ATL에서 출발한 비행기가 ABI에 도착하는 노선이 없지만 
# 출발 노선을 카테고리 타입으로 바꿔서 모든 조합을 확인하는 인덱스를 가지는 것을 볼수 있음
# 이는 카디널리티가 높은(unique 수가 많은) 카테고리 타입일수록 양이 폭발적으로 늘어남
res = (flights
    .assign(ORG_AIR=flights.ORG_AIR.astype('category'))
    .groupby(['ORG_AIR', 'DEST_AIR'])
    .agg({'CANCELLED':['sum', 'mean', 'size'],
          'AIR_TIME':['mean', 'var']})
)
res

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABI              0  NaN    0         NaN        NaN
        ABQ              0  0.0   16  170.500000  87.866667
        ABR              0  NaN    0         NaN        NaN
        ABY              0  0.0   19   28.578947   6.590643
...                    ...  ...  ...         ...        ...
SFO     TYS              0  NaN    0         NaN        NaN
        VLD              0  NaN    0         NaN        NaN
        VPS              0  NaN    0         NaN        NaN
        XNA              0  0.0    2  173.500000   0.500000
        YUM              0  NaN    0         NaN        NaN

[2710 rows x 5 columns]

In [18]:
# 이를 막기 위해 observe=True 매개변수를 사용하면 
# 일반적인 문자열 형태에 대한 그룹화처럼 작동하여 실제로 관측이 가능한 값만 표시된다.
res = (flights
    .assign(ORG_AIR=flights.ORG_AIR.astype('category'))
    .groupby(['ORG_AIR', 'DEST_AIR'], observed=True)
    .agg({'CANCELLED':['sum', 'mean', 'size'],
          'AIR_TIME':['mean', 'var']})
)
res

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
LAX     ABQ              1  0.018182   55   89.259259   29.403215
        ANC              0  0.000000    7  307.428571   78.952381
        ASE              1  0.038462   26  102.920000  102.243333
        ATL              0  0.000000  174  224.201149  127.155837
        AUS              0  0.000000   80  150.537500   57.897310
...                    ...       ...  ...         ...         ...
MSP     TTN              1  0.125000    8  124.428571   57.952381
        TUL              0  0.000000   18   91.611111   63.075163
        TUS              0  0.000000    2  176.000000   32.000000
        TVC              0  0.000000    5   56.600000   10.300000
        XNA              0  0.000000   14   90.642857  115.939560

[1130 rows x 5 columns]

## Removing the MultiIndex after grouping

In [19]:
# 요렇게 다중 인덱스가 발생하면 처리하기 어려움
# Descriptive한 인덱스를 만들어야한다
flights = pd.read_csv('data/flights.csv')
airline_info = (flights
    .groupby(['AIRLINE', 'WEEKDAY'])
    .agg({'DIST':['sum', 'mean'],
          'ARR_DELAY':['min', 'max']}) 
    .astype(int)
)
airline_info

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732
...                  ...   ...       ...  ...
WN      3         997213   782       -38  262
        4        1024854   810       -52  284
        5         981036   816       -44  244
        6         823946   834       -41  290
        7         945679   819       -45  261

[98 rows x 4 columns]

In [20]:
# get_level_values 메서드로 한 레벨씩 읽어낼수 있음
airline_info.columns.get_level_values(0)

Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object')

In [21]:
airline_info.columns.get_level_values(1)

Index(['sum', 'mean', 'min', 'max'], dtype='object')

In [22]:
# 앞에서 했던 내용 반복
# 다층 구조로 된 열을 풀 떄는 to_flat_index() 쓰면 좋다
airline_info.columns.to_flat_index()

Index([('DIST', 'sum'), ('DIST', 'mean'), ('ARR_DELAY', 'min'),
       ('ARR_DELAY', 'max')],
      dtype='object')

In [23]:
airline_info.columns = ['_'.join(x) for x in
    airline_info.columns.to_flat_index()]
airline_info

DIST_sum  DIST_mean  ARR_DELAY_min  ARR_DELAY_max
AIRLINE WEEKDAY                                                   
AA      1         1455386       1139          -60            551  
        2         1358256       1107          -52            725  
        3         1496665       1117          -45            473  
        4         1452394       1089          -46            349  
        5         1427749       1122          -41            732  
...                   ...        ...          ...            ...  
WN      3          997213        782          -38            262  
        4         1024854        810          -52            284  
        5          981036        816          -44            244  
        6          823946        834          -41            290  
        7          945679        819          -45            261  

[98 rows x 4 columns]

In [24]:
# 다층 구조의 인덱스를 풀어버리는 제일 쉬운 방법은 reset_index() 메서드를 사용하는 것
airline_info.reset_index()

,AIRLINE,WEEKDAY,DIST_sum,DIST_mean,ARR_DELAY_min,ARR_DELAY_max
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
...,...,...,...,...,...,...
93,WN,3,997213,782,-38,262
94,WN,4,1024854,810,-52,284
95,WN,5,981036,816,-44,244
96,WN,6,823946,834,-41,290


In [25]:
# .NamedAgg를 사용할 수 도 있다. 열은 이걸로 풀고 인덱스는 reset_index로 풀어준것에 주목
(flights
    .groupby(['AIRLINE', 'WEEKDAY'])
    .agg(dist_sum=pd.NamedAgg(column='DIST', aggfunc='sum'),
         dist_mean=pd.NamedAgg(column='DIST', aggfunc='mean'),
         arr_delay_min=pd.NamedAgg(column='ARR_DELAY', aggfunc='min'),
         arr_delay_max=pd.NamedAgg(column='ARR_DELAY', aggfunc='max'))
    .astype(int)
    .reset_index()
)

,AIRLINE,WEEKDAY,dist_sum,dist_mean,arr_delay_min,arr_delay_max
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
...,...,...,...,...,...,...
93,WN,3,997213,782,-38,262
94,WN,4,1024854,810,-52,284
95,WN,5,981036,816,-44,244
96,WN,6,823946,834,-41,290


In [26]:
# 멀티 인덱스를 처음부터 안 만드는 방법은 as_index 매개변수를 False로 설정하면된다.
(flights
    .groupby(['AIRLINE'], as_index=False)
    ['DIST']
    .agg('mean')
    .round(0) # 정수레벨로 반올림
)

,AIRLINE,DIST
0,AA,1114.0
1,AS,1066.0
2,B6,1772.0
3,DL,866.0
4,EV,460.0
...,...,...
9,OO,511.0
10,UA,1231.0
11,US,1181.0
12,VX,1240.0


## Grouping with a custom aggregation function

* 이미 만들어진 집계함수 외에 다른 집계를 수행해야 하는 경우가 발생할 수 있다.
* college 데이터셋 에서 주(state)당 학부생 수의 평균과 표준편차를 계산하고자 하고, 이 정보를 사용해 각 주별 평균들에서의 최대표준편차를 찾는 예시

In [27]:
# 각 주 별로 학부생 수의 평균과 표준편차를 찾음
college = pd.read_csv('data/college.csv')

In [28]:
(college
    .groupby('STABBR') # 아마도 주 
    ['UGDS'] # 아마도 Undergraduated students? 학부생 수
    .agg(['mean', 'std'])
    .round(0) 
)
# 요건 주 별 전체기관들의 학부생 수 평균과 표준편차
# 찾고자 하는건 각 기관들의 편차를 해당 주의 표준편차와 비교했을때 최고 비율을 구하고자 하는것
# 각각의 주를 표준화 시켜서 어떤 주가 최대 편차값을 가지고 있는지 비교하고자한다는것

,mean,std
STABBR,,
AK,2493.0,4052.0
AL,2790.0,4658.0
AR,1644.0,3143.0
AS,1276.0,NaN
AZ,4130.0,14894.0
...,...,...
VT,1513.0,2194.0
WA,2271.0,4124.0
WI,2655.0,4615.0


In [29]:
# 그래서 각 주별로 평균에서의 최대 표준 편차를 구하는 집계 함수 정의
def max_deviation(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()

In [30]:
# 요렇게 정의한 집계함수를 써먹을 수 있다는 것
(college
    .groupby('STABBR')
    ['UGDS']
    .agg(max_deviation)
    .round(1)
)

STABBR
AK    2.6
AL    5.8
AR    6.3
AS    NaN
AZ    9.9
     ... 
VT    3.8
WA    6.6
WI    5.8
WV    7.2
WY    2.8
Name: UGDS, Length: 59, dtype: float64

In [31]:
# 여러개의 열에다가 적용시켜도 잘 작동한다. 
# 다만 AK 주의 UGDS 처럼 수치적으로 의미가 있어야 잘동작하고 아니면 NaN
(college
    .groupby('STABBR')
    ['UGDS', 'SATVRMID', 'SATMTMID']
    .agg(max_deviation)
    .round(1)
)

<ipython-input-31-f4534dc6879b>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  (college


,UGDS,SATVRMID,SATMTMID
STABBR,,,
AK,2.6,NaN,NaN
AL,5.8,1.6,1.8
AR,6.3,2.2,2.3
AS,NaN,NaN,NaN
AZ,9.9,1.9,1.4
...,...,...,...
VT,3.8,1.9,1.9
WA,6.6,2.2,2.0
WI,5.8,2.4,2.2


In [32]:
college[college['STABBR'] == 'AK'].loc[:,['UGDS','SATVRMID','SATMTMID']]

,UGDS,SATVRMID,SATMTMID
60,12865.0,NaN,NaN
61,27.0,NaN,NaN
62,5536.0,NaN,NaN
63,1428.0,NaN,NaN
64,275.0,555.0,503.0
65,889.0,NaN,NaN
66,3256.0,NaN,NaN
67,479.0,NaN,NaN
5171,109.0,NaN,NaN
5417,68.0,NaN,NaN


In [33]:
# 기존의 집계 함수들과도 같이 쓸수 있다.
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATVRMID', 'SATMTMID'] 
    .agg([max_deviation, 'mean', 'std'])
    .round(1)
)

<ipython-input-33-490e0f49200c>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  (college


UGDS                      SATVRMID               \
                max_deviation    mean     std max_deviation   mean   std   
STABBR RELAFFIL                                                            
AK     0                 2.1   3508.9  4539.5          NaN     NaN   NaN   
       1                 1.1    123.3   132.9          NaN   555.0   NaN   
AL     0                 5.2   3248.8  5102.4          1.6   514.9  56.5   
       1                 2.4    979.7   870.8          1.5   498.0  53.0   
AR     0                 5.8   1793.7  3401.6          1.9   481.1  37.9   
...                      ...      ...     ...          ...     ...   ...   
WI     0                 5.3   2879.1  5031.5          1.3   558.8  47.5   
       1                 3.4   1716.2  1934.6          2.1   500.1  66.0   
WV     0                 6.9   1873.9  6271.7          1.6   466.7  27.9   
       1                 1.3    716.4   503.6          1.9   485.7  14.6   
WY     0                 2.8   2244.4  2744.7          NaN   535.0   NaN   

                     SATMTMID               
                max_deviation   mean   std  
STABBR RELAFFIL                             
AK     0                 NaN     NaN   NaN  
       1                 NaN   503.0   NaN  
AL     0                 1.7   515.8  56.7  
       1                 1.4   485.6  61.4  
AR     0                 2.0   503.6  39.0  
...                      ...     ...   ...  
WI     0                 1.3   591.2  85.7  
       1                 1.8   526.6  42.5  
WV     0                 1.8   480.0  27.7  
       1                 1.7   484.8  17.7  
WY     0                 NaN   540.0   NaN  

[112 rows x 9 columns]

In [34]:
# 함수 이름을 열의 이름으로 사용하게 되는데
max_deviation.__name__

'max_deviation'

In [35]:
# 그래서 rename메서드나 내부메서드를 통해 함수 이름을 바꿔주면 
# 출력 열의 이름도 바뀌는 것을 확인할 수 있다.
max_deviation.__name__ = 'Max Deviation!!!!'
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATVRMID', 'SATMTMID'] 
    .agg([max_deviation, 'mean', 'std'])
    .round(1)
)

<ipython-input-35-121fb2474e63>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  (college


UGDS                          SATVRMID         \
                Max Deviation!!!!    mean     std Max Deviation!!!!   mean   
STABBR RELAFFIL                                                              
AK     0                 2.1       3508.9  4539.5          NaN         NaN   
       1                 1.1        123.3   132.9          NaN       555.0   
AL     0                 5.2       3248.8  5102.4          1.6       514.9   
       1                 2.4        979.7   870.8          1.5       498.0   
AR     0                 5.8       1793.7  3401.6          1.9       481.1   
...                      ...          ...     ...          ...         ...   
WI     0                 5.3       2879.1  5031.5          1.3       558.8   
       1                 3.4       1716.2  1934.6          2.1       500.1   
WV     0                 6.9       1873.9  6271.7          1.6       466.7   
       1                 1.3        716.4   503.6          1.9       485.7   
WY     0                 2.8       2244.4  2744.7          NaN       535.0   

                               SATMTMID               
                  std Max Deviation!!!!   mean   std  
STABBR RELAFFIL                                       
AK     0          NaN          NaN         NaN   NaN  
       1          NaN          NaN       503.0   NaN  
AL     0         56.5          1.7       515.8  56.7  
       1         53.0          1.4       485.6  61.4  
AR     0         37.9          2.0       503.6  39.0  
...               ...          ...         ...   ...  
WI     0         47.5          1.3       591.2  85.7  
       1         66.0          1.8       526.6  42.5  
WV     0         27.9          1.8       480.0  27.7  
       1         14.6          1.7       484.8  17.7  
WY     0          NaN          NaN       540.0   NaN  

[112 rows x 9 columns]

## Customizing aggregating functions with *args and **kwargs

In [36]:
# 직접 커스텀한 집계함수에 매개변수를 필요로 할 수 도 있음
# 아래 함수는 학부생 비율이 1000~3000사이인 학교의 비율을 반환하는 함수
def pct_between_1_3k(s):
    return (s
        .between(1_000, 3_000)
        .mean()
        * 100
    )

In [37]:
(college
    .groupby(['STABBR', 'RELAFFIL'])
    ['UGDS'] 
    .agg(pct_between_1_3k)
    .round(1)
)

STABBR  RELAFFIL
AK      0           14.3
        1            0.0
AL      0           23.6
        1           33.3
AR      0           27.9
                    ... 
WI      0           13.8
        1           36.0
WV      0           24.6
        1           37.5
WY      0           54.5
Name: UGDS, Length: 112, dtype: float64

In [38]:
# 학부생 비율의 하한과 상한을 사용자가 지정할 수 있게 만든 함수
def pct_between(s, low, high):
    return s.between(low, high).mean() * 100

In [39]:
# .agg 메서드의 구조는 agg(func, *arg, **karg)
(college
    .groupby(['STABBR', 'RELAFFIL'])
    ['UGDS'] 
    .agg(pct_between, 1_000, 10_000)
    .round(1)
)

STABBR  RELAFFIL
AK      0           42.9
        1            0.0
AL      0           45.8
        1           37.5
AR      0           39.7
                    ... 
WI      0           31.0
        1           44.0
WV      0           29.2
        1           37.5
WY      0           72.7
Name: UGDS, Length: 112, dtype: float64

In [40]:
# 매개변수를 직접 열에 출력하고 싶다면 다음과 같이 파이썬의 클로져 기능을 사용할 수 있음
def between_n_m(n, m): 
    def wrapper(ser): # pct_between을 커버하는 wrapper 함수 정의
        return pct_between(ser, n, m)
    # 함수의 name 속성을 함수 이름을 매개변수를 포함하도록 지정
    wrapper.__name__ = f'between_{n}_{m}'
    # wrapper를 리턴 -> wrapper는 pct_between을 return 하므로 사실상 pct_between을 return하는것이다.
    return wrapper

In [41]:
(college
    .groupby(['STABBR', 'RELAFFIL'])
    ['UGDS'] 
    .agg([between_n_m(1_000, 10_000), 'max', 'mean'])
    .round(1)
)

between_1000_10000      max    mean
STABBR RELAFFIL                                     
AK     0                42.9         12865.0  3508.9
       1                 0.0           275.0   123.3
AL     0                45.8         29851.0  3248.8
       1                37.5          3033.0   979.7
AR     0                39.7         21405.0  1793.7
...                      ...             ...     ...
WI     0                31.0         29302.0  2879.1
       1                44.0          8212.0  1716.2
WV     0                29.2         44924.0  1873.9
       1                37.5          1375.0   716.4
WY     0                72.7          9910.0  2244.4

[112 rows x 3 columns]

## Examining the groupby object

### How to do it...

In [42]:
college = pd.read_csv('data/college.csv')
grouped = college.groupby(['STABBR', 'RELAFFIL'])
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [43]:
# groupby 객체의 속성 종류
print([attr for attr in dir(grouped) if not
    attr.startswith('_')])

['CITY', 'CURROPER', 'DISTANCEONLY', 'GRAD_DEBT_MDN_SUPP', 'HBCU', 'INSTNM', 'MD_EARN_WNE_P10', 'MENONLY', 'PCTFLOAN', 'PCTPELL', 'PPTUG_EF', 'RELAFFIL', 'SATMTMID', 'SATVRMID', 'STABBR', 'UG25ABV', 'UGDS', 'UGDS_2MOR', 'UGDS_AIAN', 'UGDS_ASIAN', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_NHPI', 'UGDS_NRA', 'UGDS_UNKN', 'UGDS_WHITE', 'WOMENONLY', 'agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'ewm', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sample', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'value_counts', 'var']


In [44]:
# groupby로 만들어진 그룹 개수
grouped.ngroups

112

In [45]:
# .groups를 통해서 각 group들을 딕셔너리 형태로 추출할 수 있다.
groups = list(grouped.groups)
len(groups[:]) # 요건 키 값들 나온거임

112

In [46]:
grouped.groups[('IL', 1)]

Int64Index([ 986,  992, 1004, 1009, 1019, 1024, 1030, 1031, 1034, 1038, 1043,
            1047, 1052, 1064, 1077, 1078, 1082, 1083, 1085, 1088, 1089, 1091,
            1092, 1095, 1096, 1097, 1105, 1106, 1107, 1114, 1119, 1129, 1131,
            1132, 1133, 1134, 1140, 1145, 1150, 1153, 1154, 1159, 1161, 1828,
            4321, 4369, 5427, 5482, 5606, 5684, 6624, 6650, 7278, 7349, 7477,
            7478, 7479],
           dtype='int64')

In [47]:
grouped.get_group(('AK', 0)) # get_group 메서드는 단일 그룹을 데이터프레임 형태로 추출

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,...,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
60,Universi...,Anchorage,AK,0.0,0.0,0.0,0,...,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5
62,Universi...,Fairbanks,AK,0.0,0.0,0.0,0,...,0.3887,1,0.2263,0.2550,0.4519,36200,19355
63,Universi...,Juneau,AK,0.0,0.0,0.0,0,...,0.5112,1,0.1769,0.1996,0.5550,37400,16875
65,AVTEC-Al...,Seward,AK,0.0,0.0,0.0,0,...,0.6817,1,0.0737,0.0664,0.7127,33500,PrivacyS...
66,Charter ...,Anchorage,AK,0.0,0.0,0.0,0,...,0.0000,1,0.8307,0.7503,0.5472,39200,13875
67,Alaska C...,Anchorage,AK,0.0,0.0,0.0,0,...,0.0000,1,0.7078,0.7860,0.5612,28700,8994
5171,Ilisagvi...,Barrow,AK,0.0,0.0,0.0,0,...,0.6239,1,0.1323,0.0000,0.6498,24900,PrivacyS...


In [48]:
# 꿀팁? 하나의 셀에서 dataframe 테이블을 여러개 출력하고 싶다면 display를 활용
from IPython.display import display
for name, group in grouped:
    print(name)
    display(group.head(3))

('AK', 0)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,...,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
60,Universi...,Anchorage,AK,0.0,0.0,0.0,0,...,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5
62,Universi...,Fairbanks,AK,0.0,0.0,0.0,0,...,0.3887,1,0.2263,0.2550,0.4519,36200,19355
63,Universi...,Juneau,AK,0.0,0.0,0.0,0,...,0.5112,1,0.1769,0.1996,0.5550,37400,16875


('AK', 1)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,...,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
61,Alaska B...,Palmer,AK,0.0,0.0,0.0,1,...,0.1481,1,0.3571,0.2857,0.4286,NaN,PrivacyS...
64,Alaska P...,Anchorage,AK,0.0,0.0,0.0,1,...,0.3745,1,0.3152,0.5297,0.4910,47000,23250
5417,Alaska C...,Soldotna,AK,0.0,0.0,0.0,1,...,0.0735,1,0.8868,0.6792,0.2264,NaN,PrivacyS...


('AL', 0)


...

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,...,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
2842,Scott Co...,Wheeling,WV,0.0,0.0,0.0,0,...,0.0000,1,0.5276,0.4961,0.1111,14800,9250
4019,B M Spur...,Glen Dale,WV,0.0,0.0,0.0,0,...,0.0000,1,0.4722,0.0000,0.4444,PrivacyS...,PrivacyS...
4020,Ben Fran...,Dunbar,WV,0.0,0.0,0.0,0,...,0.7885,1,0.1761,0.0000,0.7568,20800,PrivacyS...


('WV', 1)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,...,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
4016,Alderson...,Philippi,WV,0.0,0.0,0.0,1,...,0.0342,1,0.4800,0.8945,0.0722,46000,27000
4018,Appalach...,Mount Hope,WV,0.0,0.0,0.0,1,...,0.0612,1,0.4388,0.2770,0.0899,28700,9300
4027,Davis & ...,Elkins,WV,0.0,0.0,0.0,1,...,0.0167,1,0.4188,0.6579,0.1133,35000,23840.5


('WY', 0)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,...,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
4128,Casper C...,Casper,WY,0.0,0.0,0.0,0,...,0.3250,1,0.2188,0.1455,0.3447,34800,10764
4129,Central ...,Riverton,WY,0.0,0.0,0.0,0,...,0.4377,1,0.1700,0.0967,0.3992,25200,8757
4130,Eastern ...,Torrington,WY,0.0,0.0,0.0,0,...,0.3275,1,0.1729,0.1166,0.2371,25900,10000


In [49]:
# 요렇게만 봐도 충분하긴하다
for name, group in grouped:
    print(name)
    print(group)
    break

('AK', 0)
           INSTNM       CITY STABBR  HBCU  MENONLY  WOMENONLY  RELAFFIL  ...  \
60    Universi...  Anchorage     AK   0.0      0.0        0.0         0  ...   
62    Universi...  Fairbanks     AK   0.0      0.0        0.0         0  ...   
63    Universi...     Juneau     AK   0.0      0.0        0.0         0  ...   
65    AVTEC-Al...     Seward     AK   0.0      0.0        0.0         0  ...   
66    Charter ...  Anchorage     AK   0.0      0.0        0.0         0  ...   
67    Alaska C...  Anchorage     AK   0.0      0.0        0.0         0  ...   
5171  Ilisagvi...     Barrow     AK   0.0      0.0        0.0         0  ...   

      PPTUG_EF  CURROPER  PCTPELL  PCTFLOAN  UG25ABV  MD_EARN_WNE_P10  \
60      0.4539         1   0.2385    0.2647   0.4386        42500       
62      0.3887         1   0.2263    0.2550   0.4519        36200       
63      0.5112         1   0.1769    0.1996   0.5550        37400       
65      0.6817         1   0.0737    0.0664   0.7127     

In [50]:
group

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,...,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
60,Universi...,Anchorage,AK,0.0,0.0,0.0,0,...,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5
62,Universi...,Fairbanks,AK,0.0,0.0,0.0,0,...,0.3887,1,0.2263,0.2550,0.4519,36200,19355
63,Universi...,Juneau,AK,0.0,0.0,0.0,0,...,0.5112,1,0.1769,0.1996,0.5550,37400,16875
65,AVTEC-Al...,Seward,AK,0.0,0.0,0.0,0,...,0.6817,1,0.0737,0.0664,0.7127,33500,PrivacyS...
66,Charter ...,Anchorage,AK,0.0,0.0,0.0,0,...,0.0000,1,0.8307,0.7503,0.5472,39200,13875
67,Alaska C...,Anchorage,AK,0.0,0.0,0.0,0,...,0.0000,1,0.7078,0.7860,0.5612,28700,8994
5171,Ilisagvi...,Barrow,AK,0.0,0.0,0.0,0,...,0.6239,1,0.1323,0.0000,0.6498,24900,PrivacyS...


In [51]:
# 각 그룹별 하나씩 추출해서 볼수도 있음 
grouped.head(1)

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,...,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama ...,Normal,AL,1.0,0.0,0.0,0,...,0.0656,1,0.7356,0.8284,0.1049,30300,33888
2,Amridge ...,Montgomery,AL,0.0,0.0,0.0,1,...,0.4536,1,0.6801,0.7795,0.8540,40100,23370
43,Prince I...,Elmhurst,IL,0.0,0.0,0.0,0,...,0.0000,1,0.7857,0.9375,0.6569,PrivacyS...,20992
60,Universi...,Anchorage,AK,0.0,0.0,0.0,0,...,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5
61,Alaska B...,Palmer,AK,0.0,0.0,0.0,1,...,0.1481,1,0.3571,0.2857,0.4286,NaN,PrivacyS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4561,College ...,Majuro,MH,0.0,0.0,0.0,0,...,0.2727,1,0.8923,0.0000,0.2310,PrivacyS...,PrivacyS...
5289,Pacific ...,Mangilao,GU,0.0,0.0,0.0,1,...,0.1846,1,0.9730,0.0000,0.2533,PrivacyS...,PrivacyS...
6439,Touro Un...,Henderson,NV,0.0,0.0,0.0,1,...,0.0323,1,0.0000,0.2000,0.4000,NaN,PrivacyS...
7404,Universi...,St. Croix,VI,NaN,NaN,NaN,1,...,NaN,1,NaN,NaN,NaN,31800,15150


In [52]:
# nth 메서드를 사용하면 각 그룹의 특정 행을 선택할 수 있음 
grouped.nth([1, -1])

INSTNM         CITY  HBCU  MENONLY  WOMENONLY  SATVRMID  \
STABBR RELAFFIL                                                                 
AK     0         Universi...    Fairbanks   0.0      0.0        0.0       NaN   
       0         Ilisagvi...       Barrow   0.0      0.0        0.0       NaN   
       1         Alaska P...    Anchorage   0.0      0.0        0.0     555.0   
       1         Alaska C...     Soldotna   0.0      0.0        0.0       NaN   
AL     0         Universi...   Birmingham   0.0      0.0        0.0     570.0   
...                      ...          ...   ...      ...        ...       ...   
WV     0         BridgeVa...  South  C...   0.0      0.0        0.0       NaN   
       1         Appalach...   Mount Hope   0.0      0.0        0.0     514.0   
       1         West Vir...  Nutter Fort   NaN      NaN        NaN       NaN   
WY     0         Central ...     Riverton   0.0      0.0        0.0       NaN   
       0         CollegeA...     Cheyenne   0.0      0.0        0.0       NaN   

                 SATMTMID  ...  PPTUG_EF  CURROPER  PCTPELL  PCTFLOAN  \
STABBR RELAFFIL            ...                                          
AK     0              NaN  ...    0.3887         1   0.2263    0.2550   
       0              NaN  ...    0.6239         1   0.1323    0.0000   
       1            503.0  ...    0.3745         1   0.3152    0.5297   
       1              NaN  ...    0.0735         1   0.8868    0.6792   
AL     0            565.0  ...    0.2607         1   0.3460    0.5214   
...                   ...  ...       ...       ...      ...       ...   
WV     0              NaN  ...    0.3890         1      NaN       NaN   
       1            487.0  ...    0.0612         1   0.4388    0.2770   
       1              NaN  ...       NaN         1      NaN       NaN   
WY     0              NaN  ...    0.4377         1   0.1700    0.0967   
       0              NaN  ...    0.0000         1   0.8519    0.8642   

                 UG25ABV  MD_EARN_WNE_P10  GRAD_DEBT_MDN_SUPP  
STABBR RELAFFIL                                                
AK     0          0.4519        36200            19355         
       0          0.6498        24900      PrivacyS...         
       1          0.4910        47000            23250         
       1          0.2264          NaN      PrivacyS...         
AL     0          0.2422        39700          21941.5         
...                  ...          ...              ...         
WV     0             NaN          NaN           9429.5         
       1          0.0899        28700             9300         
       1             NaN        16700            19258         
WY     0          0.3992        25200             8757         
       0          0.7273        25600          27235.5         

[213 rows x 25 columns]

## Filtering for states with a minority majority

In [53]:
college = pd.read_csv('data/college.csv', index_col='INSTNM')
grouped = college.groupby('STABBR')
grouped.ngroups
# 그룹 갯수는 STABBR의 nunique()와 같아야함

59

In [54]:
college['STABBR'].nunique() 

59

In [55]:
def check_minority(df, threshold):
    minority_pct = 1 - df['UGDS_WHITE'] # 유색 인종 비율
    total_minority = (df['UGDS'] * minority_pct).sum() # 각 기관들 학부생 중 유색인종 수의 총합
    total_ugds = df['UGDS'].sum() # 각 기관들의 학부생 수의 총합
    total_minority_pct = total_minority / total_ugds # 전체 모든 학부생들 중 유색 인종의 비율 
    return total_minority_pct > threshold

In [56]:
minority_pct = 1 - college['UGDS_WHITE'] # 유색 인종 비율
minority_pct

INSTNM
Alabama A & M University                                  0.9667
University of Alabama at Birmingham                       0.4078
Amridge University                                        0.7010
University of Alabama in Huntsville                       0.3012
Alabama State University                                  0.9842
                                                           ...  
SAE Institute of Technology  San Francisco                   NaN
Rasmussen College - Overland Park                            NaN
National Personal Training Institute of Cleveland            NaN
Bay Area Medical Academy - San Jose Satellite Location       NaN
Excel Learning Center-San Antonio South                      NaN
Name: UGDS_WHITE, Length: 7535, dtype: float64

In [57]:
total_minority = (college['UGDS'] * minority_pct).sum() # 각 기관들 학부생 중 유색인종 수의 총합
total_minority

7898501.594700001

In [58]:
total_ugds = college['UGDS'].sum() # 각 기관들의 학부생 수의 총합
total_ugds

16200904.0

In [59]:
total_minority_pct = total_minority / total_ugds # 전체 모든 학부생들 중 유색 인종의 비율 
total_minority_pct

0.4875346211976814

In [60]:
# 여기서 필터의 역할은 threshold 값을 기준으로 각 그룹을 유지할지 여부를 판별
college_filtered = grouped.filter(check_minority, threshold=.5)
college_filtered

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,...,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,
Everest College-Phoenix,Phoenix,AZ,0.0,0.0,0.0,1,NaN,...,0.4749,0,0.8291,0.7151,0.6700,28600,9500
Collins College,Phoenix,AZ,0.0,0.0,0.0,0,NaN,...,0.3373,0,0.7205,0.8228,0.4764,25700,47000
Empire Beauty School-Paradise Valley,Phoenix,AZ,0.0,0.0,0.0,1,NaN,...,0.1600,0,0.6349,0.5873,0.4651,17800,9588
Empire Beauty School-Tucson,Tucson,AZ,0.0,0.0,0.0,0,NaN,...,0.2222,1,0.7962,0.6615,0.4229,18200,9833
Thunderbird School of Global Management,Glendale,AZ,0.0,0.0,0.0,0,NaN,...,1.0000,0,0.0000,0.0000,0.0000,118900,PrivacyS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WestMed College - Merced,Merced,CA,NaN,NaN,NaN,1,NaN,...,NaN,1,NaN,NaN,NaN,NaN,15623.5
Vantage College,El Paso,TX,NaN,NaN,NaN,1,NaN,...,NaN,1,NaN,NaN,NaN,NaN,9500
SAE Institute of Technology San Francisco,Emeryville,CA,NaN,NaN,NaN,1,NaN,...,NaN,1,NaN,NaN,NaN,NaN,9500


In [61]:
print('college.shape',college.shape)
print('college_filtered.shape',college_filtered.shape)

college.shape (7535, 26)
college_filtered.shape (3028, 26)


In [62]:
college['STABBR'].nunique()

59

In [63]:
college_filtered['STABBR'].nunique()

20

In [64]:
college_filtered_20 = grouped.filter(check_minority, threshold=.2)
college_filtered_20.shape

(7461, 26)

In [65]:
college_filtered_20['STABBR'].nunique()

57

In [66]:
college_filtered_70 = grouped.filter(check_minority, threshold=.7)
college_filtered_70.shape

(957, 26)

In [67]:
college_filtered_70['STABBR'].nunique()

10

## Transforming through a weight loss bet

In [68]:
# 두 사람의 체중 감소를 추적해 승자를 결정짓는 예제
weight_loss = pd.read_csv('data/weight_loss.csv')
weight_loss.query('Month == "Jan"') # weight_loss[weight_loss.Month == 'Jan']

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283
5,Amy,Jan,Week 3,189
6,Bob,Jan,Week 4,283
7,Amy,Jan,Week 4,190


In [69]:
# Series 데이터의 체중 비교 함수
def percent_loss(s):
    return ((s - s.iloc[0]) / s.iloc[0]) * 100

In [70]:
(weight_loss
    .query('Name=="Bob" and Month=="Jan"')
    ['Weight']
    .pipe(percent_loss)
)

0    0.000000
2   -1.030928
4   -2.749141
6   -2.749141
Name: Weight, dtype: float64

In [71]:
# groupby의 transform -> 각각의 그룹에 대해 apply
# 기존 인덱스 유지
(weight_loss
    .groupby(['Name', 'Month'])
    ['Weight'] 
    .transform(percent_loss)
)

0     0.000000
1     0.000000
2    -1.030928
3    -4.060914
4    -2.749141
        ...   
27   -3.529412
28   -3.065134
29   -3.529412
30   -4.214559
31   -5.294118
Name: Weight, Length: 32, dtype: float64

In [72]:
# assign -> 열추가 -> .assign(열이름 = (Series 데이터))
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Name=="Bob" and Month in ["Jan", "Feb"]')
)

,Name,Month,Week,Weight,percent_loss
0,Bob,Jan,Week 1,291,0.0
2,Bob,Jan,Week 2,288,-1.0
4,Bob,Jan,Week 3,283,-2.7
6,Bob,Jan,Week 4,283,-2.7
8,Bob,Feb,Week 1,283,0.0
10,Bob,Feb,Week 2,275,-2.8
12,Bob,Feb,Week 3,268,-5.3
14,Bob,Feb,Week 4,268,-5.3


In [73]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"') # 각 월에 마지막 주의 데이터만 가져옴
)

,Name,Month,Week,Weight,percent_loss
6,Bob,Jan,Week 4,283,-2.7
7,Amy,Jan,Week 4,190,-3.6
14,Bob,Feb,Week 4,268,-5.3
15,Amy,Feb,Week 4,173,-8.9
22,Bob,Mar,Week 4,261,-2.6
23,Amy,Mar,Week 4,170,-1.7
30,Bob,Apr,Week 4,250,-4.2
31,Amy,Apr,Week 4,161,-5.3


In [74]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss') # pivot으로 데이터 테이블 형태 변경
)

Name,Amy,Bob
Month,,
Apr,-5.3,-4.2
Feb,-8.9,-5.3
Jan,-3.6,-2.7
Mar,-1.7,-2.6


In [75]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
    .assign(winner=lambda df_:
            np.where(df_.Amy < df_.Bob, 'Amy', 'Bob'))
) # 변경한 테이블에 winner column 추가

Name,Amy,Bob,winner
Month,,,
Apr,-5.3,-4.2,Amy
Feb,-8.9,-5.3,Amy
Jan,-3.6,-2.7,Amy
Mar,-1.7,-2.6,Bob


In [77]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
    .assign(winner=lambda df_:
            np.where(df_.Amy < df_.Bob, 'Amy', 'Bob'))
    .style.highlight_min(subset = ['Amy', 'Bob'],axis=1)
) # 최소값 강조

Name,Amy,Bob,winner
Month,,,
Apr,-5.300000,-4.200000,Amy
Feb,-8.900000,-5.300000,Amy
Jan,-3.600000,-2.700000,Amy
Mar,-1.700000,-2.600000,Bob


In [78]:
# 이긴 횟수 비교
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
    .assign(winner=lambda df_:
            np.where(df_.Amy < df_.Bob, 'Amy', 'Bob'))
    .winner
    .value_counts()
)

Amy    3
Bob    1
Name: winner, dtype: int64

In [79]:
# pivot의 기능을 unstack으로 따라해본거라고 합니다.
# groupby로 이중 인덱스 만들고 unstack으로 풀어버린것
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .groupby(['Month', 'Name'])
    ['percent_loss']
    .first()
    .unstack()
)

Name,Amy,Bob
Month,,
Apr,-5.3,-4.2
Feb,-8.9,-5.3
Jan,-3.6,-2.7
Mar,-1.7,-2.6


### There's more...

In [80]:
# 요건 Categorical 데이터가 순서를 지정할 수 있다는 점을 이용해
# Month 열을 category로 만들어 데이터 순서를 부여하여 
# 최종 출력의 Month를 실제 월 순서로 만들어버린 예시
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)),
            Month=pd.Categorical(weight_loss.Month,
                  categories=['Jan', 'Feb', 'Mar', 'Apr'],
                  ordered=True))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
)

Name,Amy,Bob
Month,,
Jan,-3.6,-2.7
Feb,-8.9,-5.3
Mar,-1.7,-2.6
Apr,-5.3,-4.2


## Calculating weighted mean SAT scores per state with apply

In [81]:
# 각 그룹에 대해 계산을 수행하는 함수를 받는 4가지 메서드
# agg : 스칼라값 반환
# filter : boolean 반환
# transform : 전달된 그룹과 길이가 같은 Series, DataFrame 반환
# apply : 유연한 return 가능

# agg, transform -> 각 집계 열마다 호출
# apply 그룹당 한번만 호출 

# college 데이터셋에서 주별로 수학과 언어 SAT 점수의 가중 평균을 계산하는 예제 
# 학교별 학부생 수를 이용해 점수 가중
college = pd.read_csv('data/college.csv')
subset = ['UGDS', 'SATMTMID', 'SATVRMID']
college2 = college.dropna(subset=subset)
college.shape

(7535, 27)

In [82]:
college2.shape
# 결측치 싹다 제거

(1184, 27)

In [83]:
# 사용자 정의함수 생성, 가중평균 계산
def weighted_math_average(df):
    weighted_math = df['UGDS'] * df['SATMTMID']
    return int(weighted_math.sum() / df['UGDS'].sum())

In [84]:
# 그룹(주)별로 한번씩 함수가 호출되서 하나의 값으로 출력됨
college2.groupby('STABBR').apply(weighted_math_average)

STABBR
AK    503
AL    536
AR    529
AZ    569
CA    564
     ... 
VT    566
WA    555
WI    593
WV    500
WY    540
Length: 53, dtype: int64

In [85]:
# 모든 열에 agg 적용해보면 각 집계열에 대해 값을 반환하려고 하기 때문에 오류 발생
# 저 그룹에 각 집계열은 UGDS나 SATMTMID 같은 다른 열에 접근권한을 가지고 있는게 아니기 때문에 에러
(college2
    .groupby('STABBR')
    .agg(weighted_math_average)
)

KeyError: 'UGDS'

In [ ]:
# 마찬가지 SATMTMID로 열을 국한해도, UGDS라는 비집계열에 대한 접근 권한이 없어 에러발생
(college2
    .groupby('STABBR')
    ['SATMTMID'] 
    .agg(weighted_math_average)
)

In [86]:
# apply는 유연해서 Series를 반환해 다수의 새로운 열을 만들수도 있다
def weighted_average(df):
    weight_m = df['UGDS'] * df['SATMTMID']
    weight_v = df['UGDS'] * df['SATVRMID']
    wm_avg = weight_m.sum() / df['UGDS'].sum()
    wv_avg = weight_v.sum() / df['UGDS'].sum()
    data = {'w_math_avg': wm_avg,
           'w_verbal_avg': wv_avg,
           'math_avg': df['SATMTMID'].mean(),
           'verbal_avg': df['SATVRMID'].mean(),
           'count': len(df)
    }
    return pd.Series(data)


(college2
    .groupby('STABBR')
    .apply(weighted_average)
    .astype(int)
)

,w_math_avg,w_verbal_avg,math_avg,verbal_avg,count
STABBR,,,,,
AK,503,555,503,555,1
AL,536,533,504,508,21
AR,529,504,515,491,16
AZ,569,557,536,538,6
CA,564,539,562,549,72
...,...,...,...,...,...
VT,566,564,526,527,8
WA,555,541,551,548,18
WI,593,556,545,516,14


In [87]:
(college
    .groupby('STABBR')
    .apply(weighted_average)
)
# 이 예제에서 결측치를 제거하지 않으면 가중평균계산이 중단되어 잘못된 값을 출력하게 된다.

,w_math_avg,w_verbal_avg,math_avg,verbal_avg,count
STABBR,,,,,
AK,5.548091,6.121651,503.000000,555.000000,10.0
AL,261.895658,260.550109,504.285714,508.476190,96.0
AR,301.054792,287.264872,515.937500,491.875000,86.0
AS,0.000000,0.000000,NaN,NaN,1.0
AZ,61.815821,60.511712,536.666667,538.333333,133.0
...,...,...,...,...,...
VT,389.967094,388.696848,526.875000,527.500000,27.0
WA,274.885878,267.880280,551.222222,548.333333,123.0
WI,153.803086,144.160115,545.071429,516.857143,112.0


In [88]:
from scipy.stats import gmean, hmean
def calculate_means(df):
    df_means = pd.DataFrame(index=['Arithmetic', 'Weighted',
                                   'Geometric', 'Harmonic'])
    cols = ['SATMTMID', 'SATVRMID']
    for col in cols:
        arithmetic = df[col].mean()
        weighted = np.average(df[col], weights=df['UGDS'])
        geometric = gmean(df[col])
        harmonic = hmean(df[col])
        df_means[col] = [arithmetic, weighted,
                         geometric, harmonic]
    df_means['count'] = len(df)
    return df_means.astype(int)

# 평균, 가중평균, 기하평균, 조화평균에 대해 계산하는 사용자 정의함수를 
# apply를 통해 각 주별로 적용시킨 예시
(college2
    .groupby('STABBR')
    .apply(calculate_means)
)

SATMTMID  SATVRMID  count
STABBR                                      
AK     Arithmetic       503       555      1
       Weighted         503       555      1
       Geometric        503       555      1
       Harmonic         503       555      1
AL     Arithmetic       504       508     21
...                     ...       ...    ...
WV     Harmonic         480       472     17
WY     Arithmetic       540       535      1
       Weighted         540       535      1
       Geometric        540       534      1
       Harmonic         540       535      1

[212 rows x 3 columns]

## Grouping by continuous variables

In [89]:
# 이산(discrete) 값이 아닌 연속(continuous)한 값을 그룹화 하는 방법에 대한 예시
# 다양한 운항거리에 대한 항공사의 분포를 구하는 예시

flights = pd.read_csv('data/flights.csv')
flights

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58487,12,31,4,AA,SFO,DFW,515,5.0,166.0,1464,1045,-19.0,0,0
58488,12,31,4,F9,LAS,SFO,1910,13.0,71.0,414,2050,4.0,0,0
58489,12,31,4,OO,SFO,SBA,1846,-6.0,46.0,262,1956,-5.0,0,0
58490,12,31,4,WN,MSP,ATL,525,39.0,124.0,907,855,34.0,0,0


In [90]:
# 거리 값을 구분하기 위한 기준 구간 값 설정
bins = [-np.inf, 200, 500, 1000, 2000, np.inf] 
# flights['DIST']를 bins에 따라 5개의 구간으로 카테고리형 변환
cuts = pd.cut(flights['DIST'], bins=bins)
cuts

0        (500.0, ...
1        (1000.0,...
2        (500.0, ...
3        (1000.0,...
4        (1000.0,...
            ...     
58487    (1000.0,...
58488    (200.0, ...
58489    (200.0, ...
58490    (500.0, ...
58491    (500.0, ...
Name: DIST, Length: 58492, dtype: category
Categories (5, interval[float64, right]): [(-inf, 2... < (200.0, ... < (500.0, ... < (1000.0,... < (2000.0,...]

In [91]:
cuts.value_counts()

(500.0, 1000.0]     20659
(200.0, 500.0]      15874
(1000.0, 2000.0]    14186
(2000.0, inf]        4054
(-inf, 200.0]        3719
Name: DIST, dtype: int64

In [92]:
# 각 거리 구간으로 그룹을 만들고 전체 항공편 중 항공사별 비중 계산 
(flights
    .groupby(cuts)
    ['AIRLINE']
    .value_counts(normalize=True) 
    .round(3)
)

DIST           AIRLINE
(-inf, 200.0]  OO         0.326
               EV         0.289
               MQ         0.211
               DL         0.086
               AA         0.052
                          ...  
(2000.0, inf]  WN         0.046
               HA         0.028
               NK         0.019
               AS         0.012
               F9         0.004
Name: AIRLINE, Length: 57, dtype: float64

In [93]:
# 요렇게 하면 거리 그룹별  백분위별 비행시간을 찾을 수 있음 
(flights
  .groupby(cuts)
  ['AIR_TIME']
  .quantile(q=[.25, .5, .75]) 
  .div(60)
  .round(2)
)

DIST                  
(-inf, 200.0]     0.25    0.43
                  0.50    0.50
                  0.75    0.57
(200.0, 500.0]    0.25    0.77
                  0.50    0.92
                          ... 
(1000.0, 2000.0]  0.50    2.93
                  0.75    3.40
(2000.0, inf]     0.25    4.30
                  0.50    4.70
                  0.75    5.03
Name: AIR_TIME, Length: 15, dtype: float64

In [94]:
# 예시가 맞나..? cut은 거리로 해두고 라벨은 시간으로 했는데... 
# 암튼 이렇게 구간에 라벨을 부여할수도 있음. 본예시는 unstack으로 펼치기까지 함
labels=['Under an Hour', '1 Hour', '1-2 Hours',
        '2-4 Hours', '4+ Hours']
cuts2 = pd.cut(flights['DIST'], bins=bins, labels=labels)
(flights
   .groupby(cuts2)
   ['AIRLINE']
   .value_counts(normalize=True) 
   .round(3) 
   .unstack() 
)

AIRLINE,AA,AS,B6,DL,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
DIST,,,,,,,,,,,,,,
Under an Hour,0.052,NaN,NaN,0.086,0.289,NaN,NaN,0.211,NaN,0.326,0.027,NaN,NaN,0.009
1 Hour,0.071,0.001,0.007,0.189,0.156,0.005,NaN,0.100,0.012,0.159,0.062,0.016,0.028,0.194
1-2 Hours,0.144,0.023,0.003,0.206,0.101,0.038,NaN,0.051,0.030,0.106,0.131,0.025,0.004,0.138
2-4 Hours,0.264,0.016,0.003,0.165,0.016,0.031,NaN,0.003,0.045,0.046,0.199,0.040,0.012,0.160
4+ Hours,0.212,0.012,0.080,0.171,NaN,0.004,0.028,NaN,0.019,NaN,0.289,0.065,0.074,0.046


## Counting the total number of flights between cities

In [95]:
# 두 도시 사이의 총 비행횟수를 세는 예시 
flights = pd.read_csv('data/flights.csv')
flights_ct = flights.groupby(['ORG_AIR', 'DEST_AIR']).size()
# 요로코롬 출발지와 도착지에 대해 그룹화 시키고 size로 몇회인지 반환
flights_ct

ORG_AIR  DEST_AIR
ATL      ABE          31
         ABQ          16
         ABY          19
         ACY           6
         AEX          40
                    ... 
SFO      SNA         122
         STL          20
         SUN          10
         TUS          20
         XNA           2
Length: 1130, dtype: int64

In [96]:
# 이렇게도 할수 있지만... 모든 항공편에 적용할수 있는 해법이 필요
flights_ct.loc[[('ATL', 'IAH'), ('IAH', 'ATL')]]

ORG_AIR  DEST_AIR
ATL      IAH         121
IAH      ATL         148
dtype: int64

In [97]:
# 각 행의 출발지와 목적지 열을 알파벳 순으로 정렬
# 요렇게하면 열의 의미가 없어지고 두 열의 값이 각 행에대해 정렬되버림
f_part3 = (flights  # doctest: +SKIP
  [['ORG_AIR', 'DEST_AIR']] 
  .apply(lambda ser:
         ser.sort_values().reset_index(drop=True),
         axis='columns')
)
f_part3

,0,1
0,LAX,SLC
1,DEN,IAD
2,DFW,VPS
3,DCA,DFW
4,LAX,MCI
...,...,...
58487,DFW,SFO
58488,LAS,SFO
58489,SBA,SFO
58490,ATL,MSP


In [98]:
# 열이름 재지정해주고
# 다시 그룹화 시켜봄
rename_dict = {0:'AIR1', 1:'AIR2'}  
(flights     # doctest: +SKIP
  [['ORG_AIR', 'DEST_AIR']]
  .apply(lambda ser:
         ser.sort_values().reset_index(drop=True),
         axis='columns')
  .rename(columns=rename_dict)
  .groupby(['AIR1', 'AIR2'])
  .size()
)

AIR1  AIR2
ABE   ATL      31
      ORD      24
ABI   DFW      74
ABQ   ATL      16
      DEN      46
             ... 
SFO   SNA     122
      STL      20
      SUN      10
      TUS      20
      XNA       2
Length: 1085, dtype: int64

In [99]:
# 
(flights     # doctest: +SKIP
  [['ORG_AIR', 'DEST_AIR']]
  .apply(lambda ser:
         ser.sort_values().reset_index(drop=True),
         axis='columns') #
  .rename(columns=rename_dict)
  .groupby(['AIR1', 'AIR2'])
  .size()
  .loc[('ATL', 'IAH')]
)

269

In [92]:
# 행 별로 정렬되어있어 순서를 바꾸면 오류가 나게 된다.(해당 키가 없다고)
(flights     # doctest: +SKIP
  [['ORG_AIR', 'DEST_AIR']]
  .apply(lambda ser:
         ser.sort_values().reset_index(drop=True),
         axis='columns')
  .rename(columns=rename_dict)
  .groupby(['AIR1', 'AIR2'])
  .size()
  .loc[('IAH', 'ATL')]
)

KeyError: ('IAH', 'ATL')

### There's more ...

In [108]:
# axis = 'columns'로 .apply 호출하는 것은 성능이 낮은 작업으로 더 큰데이터에서 사용은 힘듬
# Numpy sort를 사용하면 속도를 크게 올릴수 있다는 예시
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
data_sorted[:10]

array([['LAX', 'SLC'],
       ['DEN', 'IAD'],
       ['DFW', 'VPS'],
       ['DCA', 'DFW'],
       ['LAX', 'MCI'],
       ['IAH', 'SAN'],
       ['DFW', 'MSY'],
       ['PHX', 'SFO'],
       ['ORD', 'STL'],
       ['IAH', 'SJC']], dtype=object)

In [109]:
flights_sort2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])
flights_sort2.equals(f_part3.rename(columns={'ORG_AIR':'AIR1',
    'DEST_AIR':'AIR2'}))

False

In [110]:
%%timeit
flights_sort = (flights   # doctest: +SKIP
    [['ORG_AIR', 'DEST_AIR']] 
   .apply(lambda ser:
         ser.sort_values().reset_index(drop=True),
         axis='columns')
)

9.72 s ± 68.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [111]:
%%timeit
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
flights_sort2 = pd.DataFrame(data_sorted,
    columns=['AIR1', 'AIR2'])

7.77 ms ± 305 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Finding the longest streak of on-time flights

In [96]:
# 각 출발지에서 항공사 별로 가장 긴 연속 정시 비행을 찾는 예시
#요건 짧은 예시
s = pd.Series([0, 1, 1, 0, 1, 1, 1, 0])
s

0    0
1    1
2    1
3    0
4    1
5    1
6    1
7    0
dtype: int64

In [97]:
# .cumsum() 누적합
s1 = s.cumsum()
s1

0    0
1    1
2    2
3    2
4    3
5    4
6    5
7    5
dtype: int64

In [98]:
# 누적합과 기존데이터간의 곱
s.mul(s1)

0    0
1    1
2    2
3    0
4    3
5    4
6    5
7    0
dtype: int64

In [99]:
# 각 인덱스간의 차이
s.mul(s1).diff()

0    NaN
1    1.0
2    1.0
3   -2.0
4    3.0
5    1.0
6    1.0
7   -5.0
dtype: float64

In [100]:
# 저 차이에서 음수는 연속된 1이 끝났음을 의미
# 끝난 지점 외에는 죄다 nan 처리
(s
    .mul(s.cumsum())
    .diff()
    .where(lambda x: x < 0)
)

0    NaN
1    NaN
2    NaN
3   -2.0
4    NaN
5    NaN
6    NaN
7   -5.0
dtype: float64

In [101]:
# ffill을 쓰면 아래로 값을 전파
(s
    .mul(s.cumsum())
    .diff()
    .where(lambda x: x < 0)
    .ffill()
)

0    NaN
1    NaN
2    NaN
3   -2.0
4   -2.0
5   -2.0
6   -2.0
7   -5.0
dtype: float64

In [102]:
# 전파한 값에 누적합산을 더하면 초과누적 상쇄
(s
    .mul(s.cumsum())
    .diff()
    .where(lambda x: x < 0)
    .ffill()
    .add(s.cumsum(), fill_value=0)
)

0    0.0
1    1.0
2    2.0
3    0.0
4    1.0
5    2.0
6    3.0
7    0.0
dtype: float64

In [103]:
# 이제 본 예제 시작
flights = pd.read_csv('data/flights.csv')
(flights
    .assign(ON_TIME=flights['ARR_DELAY'].lt(15).astype(int))
    [['AIRLINE', 'ORG_AIR', 'ON_TIME']]
)

,AIRLINE,ORG_AIR,ON_TIME
0,WN,LAX,0
1,UA,DEN,1
2,MQ,DFW,0
3,AA,DFW,1
4,WN,LAX,0
...,...,...,...
58487,AA,SFO,1
58488,F9,LAS,1
58489,OO,SFO,1
58490,WN,MSP,0


In [104]:
# 위에서 했던 예시를 고대로 적용해서 Series의 최대 연속을 반환하는 함수를 정의

def max_streak(s):
    s1 = s.cumsum()
    return (s
       .mul(s1)
       .diff()
       .where(lambda x: x < 0) 
       .ffill()
       .add(s1, fill_value=0)
       .max()
    )

In [105]:
# 항공사와 출발지 공항별 정시도착비율, 비행횟수, 최대연속 정시 비행횟수 구하는 코드
(flights
    .assign(ON_TIME=flights['ARR_DELAY'].lt(15).astype(int))
    .sort_values(['MONTH', 'DAY', 'SCHED_DEP']) 
    .groupby(['AIRLINE', 'ORG_AIR'])
    ['ON_TIME'] 
    .agg(['mean', 'size', max_streak])
    .round(2)
)

mean  size  max_streak
AIRLINE ORG_AIR                        
AA      ATL      0.82   233          15
        DEN      0.74   219          17
        DFW      0.78  4006          64
        IAH      0.80   196          24
        LAS      0.79   374          29
...               ...   ...         ...
WN      LAS      0.77  2031          39
        LAX      0.70  1135          23
        MSP      0.84   237          32
        PHX      0.77  1724          33
        SFO      0.76   445          17

[114 rows x 3 columns]

### There's more...

In [106]:
# 가장 긴 연착 찾기
# Ontime 아닌거 찾아서 똑같이 돌리면 됩니다.. 
def max_delay_streak(df):
    df = df.reset_index(drop=True)
    late = 1 - df['ON_TIME']
    late_sum = late.cumsum()
    streak = (late
        .mul(late_sum)
        .diff()
        .where(lambda x: x < 0) 
        .ffill()
        .add(late_sum, fill_value=0)
    )
    last_idx = streak.idxmax()
    first_idx = last_idx - streak.max() + 1
    res = (df
        .loc[[first_idx, last_idx], ['MONTH', 'DAY']]
        .assign(streak=streak.max())
    )
    res.index = ['first', 'last']
    return res

In [110]:
(flights
    .assign(ON_TIME=flights['ARR_DELAY'].lt(15).astype(int))
    .sort_values(['MONTH', 'DAY', 'SCHED_DEP']) 
    .groupby(['AIRLINE', 'ORG_AIR']) 
    .apply(max_delay_streak) 
    .sort_values('streak', ascending=False)
)

MONTH   DAY  streak
AIRLINE ORG_AIR                           
AA      DFW     first    2.0  26.0    38.0
                last     3.0   1.0    38.0
MQ      ORD     last     1.0  12.0    28.0
                first    1.0   6.0    28.0
        DFW     last     2.0  26.0    25.0
...                      ...   ...     ...
US      LAS     last     1.0   7.0     1.0
AS      ATL     first    5.0   4.0     1.0
OO      LAS     first    2.0   8.0     1.0
EV      PHX     last     8.0   1.0     0.0
                first    NaN   NaN     0.0

[228 rows x 3 columns]